In [1]:
from ultralytics import YOLO
import cv2
import numpy as np

model = YOLO(r"runs\detect\train\weights\best.pt")   # your trained model
image_path = r"test\images\14_021_1_0093_bmp.rf.c694c5ccdd106e9718a72ac7f8579880.jpg"

img = cv2.imread(image_path)
H, W = img.shape[:2]

results = model(img, conf=0.05,device=0)



0: 640x640 1 Stenosis, 8.0ms
Speed: 4.0ms preprocess, 8.0ms inference, 70.9ms postprocess per image at shape (1, 3, 640, 640)


In [2]:
result = results[0]

# assuming single stenosis detection
box = result.boxes.xyxy[0].cpu().numpy()
conf = result.boxes.conf[0].item()

x1, y1, x2, y2 = map(int, box)


In [3]:
scale = 3.0  # EXPAND aggressively

cx = (x1 + x2) // 2
cy = (y1 + y2) // 2

w = int((x2 - x1) * scale)
h = int((y2 - y1) * scale)

x1_p = max(0, cx - w)
y1_p = max(0, cy - h)
x2_p = min(W, cx + w)
y2_p = min(H, cy + h)

roi = img[y1_p:y2_p, x1_p:x2_p]


In [4]:
roi = img[y1_p:y2_p, x1_p:x2_p]


In [5]:
cv2.imwrite("roi_stenosis.png", roi)

cv2.imshow("ROI", roi)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [6]:
sx1 = x1 - x1_p
sy1 = y1 - y1_p
sx2 = x2 - x1_p
sy2 = y2 - y1_p


In [7]:
roi_h, roi_w = roi.shape[:2]

sx1 = max(0, min(sx1, roi_w - 1))
sx2 = max(0, min(sx2, roi_w - 1))
sy1 = max(0, min(sy1, roi_h - 1))
sy2 = max(0, min(sy2, roi_h - 1))


In [8]:
import json

meta = {
    "roi_image": "roi_stenosis.png",
    "yolo_box": [int(sx1), int(sy1), int(sx2), int(sy2)],
    "confidence": float(conf)
}

with open("roi_stenosis.json", "w") as f:
    json.dump(meta, f, indent=2)


In [12]:
roi_vis = roi.copy()
cv2.rectangle(roi_vis, (sx1, sy1), (sx2, sy2), (255, 0, 0), 2)

cv2.imshow("ROI + YOLO Box", roi_vis)
cv2.waitKey(0)
cv2.destroyAllWindows()
